<a href="https://colab.research.google.com/github/Nikita-devel/DS_Hw7/blob/main/DS_Hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-surprise
import pandas as pd
from surprise import SVD, SVDpp, NMF
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

# Завантаження датасету movielens
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [3]:
df = pd.DataFrame(data.raw_ratings, columns=["user_id", "item_id", "rating", "timestamp"])
print(df.head())

  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596


In [4]:
svd_model = SVD()
svdpp_model = SVDpp()
nmf_model = NMF()

# Крос-валідація
cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(svdpp_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(nmf_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9419  0.9373  0.9319  0.9296  0.9376  0.9356  0.0044  
MAE (testset)     0.7425  0.7374  0.7341  0.7303  0.7392  0.7367  0.0042  
Fit time          1.69    1.21    1.43    1.48    1.23    1.41    0.18    
Test time         0.13    0.11    0.18    0.14    0.11    0.13    0.03    
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9177  0.9252  0.9143  0.9223  0.9190  0.9197  0.0038  
MAE (testset)     0.7204  0.7281  0.7166  0.7252  0.7217  0.7224  0.0040  
Fit time          26.02   26.69   26.47   26.18   26.22   26.32   0.23    
Test time         6.09    4.41    5.06    4.33    5.16    5.01    0.63    
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (tests

{'test_rmse': array([0.96271982, 0.96315368, 0.94963546, 0.97070567, 0.96647243]),
 'test_mae': array([0.75920911, 0.75690024, 0.74770915, 0.76160118, 0.75799451]),
 'fit_time': (1.7901103496551514,
  1.6936166286468506,
  1.662046194076538,
  1.8278968334197998,
  2.244724750518799),
 'test_time': (0.09181928634643555,
  0.19760346412658691,
  0.09769058227539062,
  0.29086875915527344,
  0.09203720092773438)}

In [9]:
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.4, 0.6, 0.8]}

# Використання GridSearchCV для пошуку оптимальних параметрів
grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE'], cv=5)
grid_search.fit(data)

In [10]:
best_params = grid_search.best_params['rmse']
print("Best Parameters:", best_params)

Best Parameters: {'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.4}


In [11]:
final_svd_model = SVD(n_epochs=best_params['n_epochs'], lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])
trainset = data.build_full_trainset()
final_svd_model.fit(trainset)

In [ ]:
user_id = '42'
all_predictions = final_svd_model.test(trainset.build_anti_testset(fill=None))

# Виведення всіх прогнозів для користувача
for pred in all_predictions:
    print(f"Item ID: {pred.iid}, Prediction: {pred.est}")
#Краще не запускати...

In [3]:
import numpy as np

In [4]:
class MatrixFactorization:
    def __init__(self, num_users, num_items, num_features, learning_rate, reg_param, num_iterations):
        self.num_users = num_users
        self.num_items = num_items
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.num_iterations = num_iterations
        self.user_matrix = np.random.rand(num_users, num_features)
        self.item_matrix = np.random.rand(num_items, num_features)

    def predict_rating(self, user_id, item_id):
        return np.dot(self.user_matrix[user_id], self.item_matrix[item_id])

    def loss_function(self, user_id, item_id, true_rating):
        predicted_rating = self.predict_rating(user_id, item_id)
        error = true_rating - predicted_rating
        regularization_term = 0.5 * (np.linalg.norm(self.user_matrix[user_id]) + np.linalg.norm(self.item_matrix[item_id]))
        return 0.5 * error**2 + 0.5 * self.reg_param * regularization_term

    def gradient_descent(self, user_id, item_id, true_rating):
        error = true_rating - self.predict_rating(user_id, item_id)
        user_gradient = -error * self.item_matrix[item_id] + self.reg_param * self.user_matrix[user_id]
        item_gradient = -error * self.user_matrix[user_id] + self.reg_param * self.item_matrix[item_id]

        self.user_matrix[user_id] -= self.learning_rate * user_gradient
        self.item_matrix[item_id] -= self.learning_rate * item_gradient

    def train(self, user_ratings):
        for _ in range(self.num_iterations):
            for user_id, item_id, rating in user_ratings:
                self.gradient_descent(user_id, item_id, rating)

    def evaluate(self, test_ratings):
        total_loss = 0.0
        for user_id, item_id, true_rating in test_ratings:
            total_loss += self.loss_function(user_id, item_id, true_rating)
        return total_loss / len(test_ratings)

In [6]:
num_users = 100
num_items = 50
num_features = 10
learning_rate = 0.01
reg_param = 0.01
num_iterations = 50

mf_model = MatrixFactorization(num_users, num_items, num_features, learning_rate, reg_param, num_iterations)

In [7]:
np.random.seed(42)
train_ratings = np.random.randint(0, 6, size=(500, 3))  # припустимо, що є 500 рейтингів у тренувальному датасеті
test_ratings = np.random.randint(0, 6, size=(100, 3))   # припустимо, що є 100 рейтингів у тестовому датасеті


In [8]:
mf_model.train(train_ratings)

In [9]:
test_loss = mf_model.evaluate(test_ratings)
print(f'Test Loss: {test_loss}')


Test Loss: 1.7745863957112649
